In [9]:
# step 1
import pandas as pd
import numpy as np

df = pd.read_csv('ms_data.csv')
df['visit_date'] = pd.to_datetime(df['visit_date'])
df.sort_values(by=['patient_id','visit_date'])
print(df)

      patient_id visit_date    age education_level  walking_speed
0          P0001 2020-01-07  53.07        Graduate           4.03
1          P0001 2020-04-19  53.36        Graduate           4.97
2          P0001 2020-07-09  53.58        Graduate           4.15
3          P0001 2020-09-26  53.79        Graduate           4.34
4          P0001 2020-12-22  54.03        Graduate           4.47
...          ...        ...    ...             ...            ...
15382      P1000 2022-05-23  75.30    Some College           2.53
15383      P1000 2022-08-29  75.57    Some College           3.60
15384      P1000 2023-03-02  76.08    Some College           3.03
15385      P1000 2023-06-12  76.36    Some College           2.66
15386      P1000 2023-09-23  76.64    Some College           2.92

[15387 rows x 5 columns]


In [19]:
# step 2
insurance = pd.read_table('insurance.lst')

id_list = np.unique(df['patient_id'])
insurance_assignment = np.random.randint(3, size=len(id_list))
df['insurance_type'] = df['patient_id']
for i in range(len(id_list)):
    number = insurance_assignment[i]
    df['insurance_type'].replace(to_replace=id_list[i], insurance[number])
print(df)

SyntaxError: positional argument follows keyword argument (574968072.py, line 9)